# Network attacks - Preparation de données

## I - Modules

In [1]:
import pandas as pd

## II - Charger donnée

In [3]:
def load_data(size):
    if size=="full":
        df_normal = pd.read_csv("network/normal.csv")
        df_attack_1 = pd.read_csv("network/attack_1.csv")
        df_attack_2 = pd.read_csv("network/attack_2.csv")
        df_attack_3 = pd.read_csv("network/attack_3.csv")
        df_attack_4 = pd.read_csv("network/attack_4.csv")
    
    elif size=="/2":
        df_normal = pd.read_csv("network/normal.csv", skiprows=lambda x: x % 2 != 0)
        df_attack_1 = pd.read_csv("network/attack_1.csv", skiprows=lambda x: x % 2 != 0)
        df_attack_2 = pd.read_csv("network/attack_2.csv", skiprows=lambda x: x % 2 != 0)
        df_attack_3 = pd.read_csv("network/attack_3.csv", skiprows=lambda x: x % 2 != 0)
        df_attack_4 = pd.read_csv("network/attack_4.csv", skiprows=lambda x: x % 2 != 0)
    
    elif size=="/4":
        df_normal = pd.read_csv("network/normal.csv", skiprows=lambda x: x % 4 != 0)
        df_attack_1 = pd.read_csv("network/attack_1.csv", skiprows=lambda x: x % 4 != 0)
        df_attack_2 = pd.read_csv("network/attack_2.csv", skiprows=lambda x: x % 4 != 0)
        df_attack_3 = pd.read_csv("network/attack_3.csv", skiprows=lambda x: x % 4 != 0)
        df_attack_4 = pd.read_csv("network/attack_4.csv", skiprows=lambda x: x % 4 != 0)

    elif size=="/10":
        df_normal = pd.read_csv("network/normal.csv", skiprows=lambda x: x % 10 != 0)
        df_attack_1 = pd.read_csv("network/attack_1.csv", skiprows=lambda x: x % 10 != 0)
        df_attack_2 = pd.read_csv("network/attack_2.csv", skiprows=lambda x: x % 10 != 0)
        df_attack_3 = pd.read_csv("network/attack_3.csv", skiprows=lambda x: x % 10 != 0)
        df_attack_4 = pd.read_csv("network/attack_4.csv", skiprows=lambda x: x % 10 != 0)
    
    elif size=="/100":
        df_normal = pd.read_csv("network/normal.csv", skiprows=lambda x: x % 100 != 0)
        df_attack_1 = pd.read_csv("network/attack_1.csv", skiprows=lambda x: x % 100 != 0)
        df_attack_2 = pd.read_csv("network/attack_2.csv", skiprows=lambda x: x % 100 != 0)
        df_attack_3 = pd.read_csv("network/attack_3.csv", skiprows=lambda x: x % 100 != 0)
        df_attack_4 = pd.read_csv("network/attack_4.csv", skiprows=lambda x: x % 100 != 0)
    
    elif size=="/1000":
        df_normal = pd.read_csv("network/normal.csv", skiprows=lambda x: x % 1000 != 0)
        df_attack_1 = pd.read_csv("network/attack_1.csv", skiprows=lambda x: x % 1000 != 0)
        df_attack_2 = pd.read_csv("network/attack_2.csv", skiprows=lambda x: x % 1000 != 0)
        df_attack_3 = pd.read_csv("network/attack_3.csv", skiprows=lambda x: x % 1000 != 0)
        df_attack_4 = pd.read_csv("network/attack_4.csv", skiprows=lambda x: x % 1000 != 0)


    
    return df_normal, df_attack_1, df_attack_2, df_attack_3, df_attack_4

In [5]:
load_data("/10")

## III - Nettoyer donnée

Nous allons nettoyer la donnée avec les informations obtenus à partir de l'exploration

### 1 - Supprimer espaces superflus

Certaines colonnes sont écrites " sport" et d'autres "sport", supprimons les espaces.

In [10]:
def del_espace(string):
    ret = ""
    for i in range(len(string)):
        if string[i]!=" ":
            ret+=string[i]
    return ret


In [11]:
def clean_espace(df):
    df_ret = df.copy()
    for col in df_ret.columns:
        df_ret = df_ret.rename(columns={col:del_espace(col)})
    return df_ret

In [12]:
df_attack_1.columns

Index(['Time', ' mac_s', ' mac_d', ' ip_s', ' ip_d', ' sport', ' dport',
       ' proto', ' flags', ' size', ' modbus_fn', ' n_pkt_src', ' n_pkt_dst',
       ' modbus_response', ' label_n', ' label'],
      dtype='object')

In [21]:
cleaned_attack_1 = clean_espace(df_attack_1)
cleaned_attack_1.columns

Index(['Time', 'mac_s', 'mac_d', 'ip_s', 'ip_d', 'sport', 'dport', 'proto',
       'flags', 'size', 'modbus_fn', 'n_pkt_src', 'n_pkt_dst',
       'modbus_response', 'label_n', 'label'],
      dtype='object')

Cela a fonctionné. Applicons cela à tous les dataset.

In [22]:
cleaned_normal = clean_espace(df_normal)
cleaned_attack_2 = clean_espace(df_attack_2)
cleaned_attack_3 = clean_espace(df_attack_3)
cleaned_attack_4 = clean_espace(df_attack_4)

Vérifions que tous les dataset ont désormais les mêmes colonnes : 

In [25]:
print(list(cleaned_normal.columns)==list(cleaned_attack_2.columns) and
      list(cleaned_attack_1.columns)==list(cleaned_attack_2.columns) and 
      list(cleaned_attack_1.columns)==list(cleaned_attack_3.columns) and 
      list(cleaned_attack_1.columns)==list(cleaned_attack_3.columns))

True


### 2 - Supprimer colonnes catégorielles et temporelles

Cela peut être utile de supprimer les données catégorielles qui ne peuvent pas être prises en compte par certains algorithms

In [29]:
def clean_category(df):
    df_ret = df.drop("Time",axis=1)
    df_ret = df_ret.drop("mac_s",axis=1)
    df_ret = df_ret.drop("mac_d",axis=1)
    df_ret = df_ret.drop("ip_s",axis=1)
    df_ret = df_ret.drop("ip_d",axis=1)
    df_ret = df_ret.drop("modbus_fn",axis=1)
    df_ret = df_ret.drop("modbus_response",axis=1)
    df_ret = df_ret.drop("label_n",axis=1)
    df_ret = df_ret.drop("proto",axis=1)


    return df_ret

In [30]:
cleaned_normal = clean_category(cleaned_normal)
cleaned_attack_1 = clean_category(cleaned_attack_1)
cleaned_attack_2 = clean_category(cleaned_attack_2)
cleaned_attack_3 = clean_category(cleaned_attack_3)
cleaned_attack_4 = clean_category(cleaned_attack_4)

Vérifions :

In [31]:
cleaned_normal.head()

,sport,dport,flags,size,n_pkt_src,n_pkt_dst,label
0,61516,502,11000,66,0,0,normal
1,502,61516,11000,65,0,0,normal
2,61515,502,11000,66,1,0,normal
3,502,61515,11000,65,0,1,normal
4,502,61514,11000,64,0,2,normal


In [32]:
cleaned_attack_1.head()

,sport,dport,flags,size,n_pkt_src,n_pkt_dst,label
0,56667.0,502.0,11000.0,66,0.0,0.0,normal
1,56666.0,502.0,11000.0,66,1.0,0.0,normal
2,56668.0,502.0,11000.0,66,2.0,0.0,normal
3,502.0,56667.0,11000.0,64,0.0,0.0,normal
4,502.0,56668.0,11000.0,64,0.0,1.0,normal


### 3 - Supprimer les NaN

In [33]:
for k in cleaned_attack_1.columns:
    print(k, cleaned_attack_1[k].isna().any())

sport True
dport True
flags True
size False
n_pkt_src True
n_pkt_dst True
label False


In [34]:
cleaned_normal = cleaned_normal.dropna(subset=["sport"])
cleaned_attack_1 = cleaned_attack_1.dropna(subset=["sport"])
cleaned_attack_2 = cleaned_attack_2.dropna(subset=["sport"])
cleaned_attack_3 = cleaned_attack_3.dropna(subset=["sport"])
cleaned_attack_4 = cleaned_attack_4.dropna(subset=["sport"])

Vérifions que ça a retiré tous les NaN :

In [36]:
for k in cleaned_attack_1.columns:
    print(k, cleaned_attack_1[k].isna().any())

sport False
dport False
flags False
size False
n_pkt_src False
n_pkt_dst False
label False


## IV - Enregistrer données nettoyées

In [38]:
cleaned_normal.to_csv("preparation/network_normal.csv")
cleaned_attack_1.to_csv("preparation/network_1.csv")
cleaned_attack_2.to_csv("preparation/network_2.csv")
cleaned_attack_3.to_csv("preparation/network_3.csv")
cleaned_attack_4.to_csv("preparation/network_4.csv")